# Challenge 2. Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [2]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib as plt
# Import the API key.
from config2 import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Lebu,CL,2020-08-27 18:16:34,-37.62,-73.65,284.39,79,100,4.76,light rain,0.15,0
1,1,Ushuaia,AR,2020-08-27 18:16:35,-54.80,-68.30,278.15,38,40,10.30,shower rain,0.00,0
2,2,Scarborough,GB,2020-08-27 18:16:35,54.28,-0.40,286.48,87,99,0.45,light rain,0.48,0
3,3,Songea,TZ,2020-08-27 18:16:36,-10.68,35.65,295.15,60,20,2.10,few clouds,0.00,0
4,4,Bluff,NZ,2020-08-27 18:16:37,-46.60,168.33,279.08,80,75,5.95,broken clouds,0.00,0


In [4]:
def to_bool(s):
    return 1 if s == 'yes' else 0

In [5]:
# Get the data types
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
raining = to_bool(input("Do you want it to be raining? (yes/no) "))
snowing = to_bool(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 295
What is the maximum temperature you would like for your trip? 305
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [8]:
# Filter the dataset to find the cities that fit the criteria.
if raining == 0 and snowing == 0:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] == raining) & \
                                           (city_data_df["Snow (inches)"] == snowing)]
elif raining == 0 and snowing == 1:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] == raining) & \
                                           (city_data_df["Snow (inches)"] > 0.0) ]
elif raining == 1 and snowing == 0:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] > 0.0) & \
                                           (city_data_df["Snow (inches)"] == snowing)]
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) & \
                                           (city_data_df["Rain (inches)"] > 0.0 ) & \
                                           (city_data_df["Snow (inches)"] > 0.0)]
    
        
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
3,3,Songea,TZ,2020-08-27 18:16:36,-10.68,35.65,295.15,60,20,2.10,few clouds,0.0,0
6,6,Kamariotissa,GR,2020-08-27 18:16:38,40.48,25.47,300.15,54,30,3.10,scattered clouds,0.0,0
9,9,Bambous Virieux,MU,2020-08-27 18:16:40,-20.34,57.76,295.15,83,75,5.70,shower rain,0.0,0
11,11,Castro,BR,2020-08-27 18:16:41,-24.79,-50.01,297.01,35,0,1.31,clear sky,0.0,0
14,14,Barcelos,PT,2020-08-27 18:16:44,41.54,-8.62,297.59,94,20,5.70,few clouds,0.0,0
18,18,Lata,PT,2020-08-27 18:16:47,40.16,-8.33,298.15,75,0,2.24,clear sky,0.0,0
20,20,Bima,ID,2020-08-27 18:16:48,-8.47,118.72,295.60,83,14,1.45,few clouds,0.0,0
25,25,Tamandare,BR,2020-08-27 18:16:51,-8.76,-35.10,298.81,60,29,4.11,scattered clouds,0.0,0
27,27,Nouadhibou,MR,2020-08-27 18:16:53,20.93,-17.03,303.15,62,0,7.70,sand,0.0,0
28,28,Bengkulu,ID,2020-08-27 18:11:59,-3.80,102.27,297.49,84,2,2.66,clear sky,0.0,0


In [22]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Rain (inches)          0
Snow (inches)          0
dtype: int64

In [23]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Kamariotissa,GR,300.15,scattered clouds,40.48,25.47,
11,Castro,BR,297.01,clear sky,-24.79,-50.01,
14,Barcelos,PT,297.59,few clouds,41.54,-8.62,
18,Lata,PT,298.15,clear sky,40.16,-8.33,
25,Tamandare,BR,298.81,scattered clouds,-8.76,-35.10,
27,Nouadhibou,MR,303.15,sand,20.93,-17.03,
28,Bengkulu,ID,297.49,clear sky,-3.80,102.27,
33,San Policarpo,PH,299.79,overcast clouds,12.18,125.51,
38,Hit,IQ,304.80,clear sky,33.64,42.83,
42,Korla,CN,297.67,overcast clouds,41.76,86.15,


In [25]:
# Dependencies and Setup
import requests
import gmaps
import pprint

# Import API key
from config2 import g_key

In [27]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [28]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [54]:
hotel_df.head(10)

,index,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,6,Kamariotissa,GR,300.15,scattered clouds,40.48,25.47,Niki Beach
1,11,Castro,BR,297.01,clear sky,-24.79,-50.01,CHACARA BAILLY
2,14,Barcelos,PT,297.59,few clouds,41.54,-8.62,Hotel Do Terço
3,18,Lata,PT,298.15,clear sky,40.16,-8.33,Casa Maquia
4,25,Tamandare,BR,298.81,scattered clouds,-8.76,-35.10,Pousada 100 Passos
5,27,Nouadhibou,MR,303.15,sand,20.93,-17.03,El Medina
6,28,Bengkulu,ID,297.49,clear sky,-3.80,102.27,Grage Hotel Bengkulu
7,33,San Policarpo,PH,299.79,overcast clouds,12.18,125.51,Zal-Ann Beach Resort
8,38,Hit,IQ,304.80,clear sky,33.64,42.83,Aboody Alhachem
9,42,Korla,CN,297.67,overcast clouds,41.76,86.15,Gulisitan Hotel


In [55]:
hotel_df = hotel_df.set_index('index')

In [56]:
# Create the output file (CSV).
output_data_file = "data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index ="City ID")

In [47]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

In [48]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [57]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))